This is a starter kernel to train a YOLOv5 model on [SIIM-FISABIO-RSNA COVID-19 Detection](https://www.kaggle.com/c/siim-covid19-detection/overview) dataset. Given an input image the task is to find the region of opacity in the chest  using bounding box coordinates. Check out [Visualize Bounding Boxes Interactively](https://www.kaggle.com/ayuraj/visualize-bounding-boxes-interactively) for interactive bounding box EDA. 

## 🖼️ What is YOLOv5?

YOLO an acronym for 'You only look once', is an object detection algorithm that divides images into a grid system. Each cell in the grid is responsible for detecting objects within itself.

[Ultralytics' YOLOv5](https://ultralytics.com/yolov5) ("You Only Look Once") model family enables real-time object detection with convolutional neural networks. 

## 🦄 What is Weights and Biases?

Weights & Biases (W&B) is a set of machine learning tools that helps you build better models faster. Check out [Experiment Tracking with Weights and Biases](https://www.kaggle.com/ayuraj/experiment-tracking-with-weights-and-biases) to learn more.  
Weights & Biases is directly integrated into YOLOv5, providing experiment metric tracking, model and dataset versioning, rich model prediction visualization, and more.


It's a work in progress:

✔️ Required folder structure. <br>
✔️ Bounding box format required for YOLOv5. <br>
✔️ **Train** a small YOLOv5 model. <br>
✔️ Experiment tracking with W&B. <br>
✔️ Proper documentation <br>
✔️ Inference <br>

❌ Model prediction visualization. 

## Results 

### [Check out W&B Run Page $\rightarrow$](https://wandb.ai/ayush-thakur/kaggle-siim-covid/runs/1bk93e3j)

![img](https://i.imgur.com/quOYtNN.gif)

# ☀️ Imports and Setup

According to the official [Train Custom Data](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data) guide, YOLOv5 requires a certain directory structure. 

```
/parent_folder
    /dataset
         /images
         /labels
    /yolov5
```

* We thus will create a `/tmp` directory. <br>
* Download YOLOv5 repository and pip install the required dependencies. <br>
* Install the latest version of W&B and login with your wandb account. You can create your free W&B account [here](https://wandb.ai/site).

In [2]:
# %cd ../
# !mkdir tmp
# %cd tmp

In [5]:
# Download YOLOv5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
# # Install dependencies
# %pip install -qr requirements.txt  # install dependencies

# %cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.7.1+cu101 (GeForce RTX 2080 Ti)


In [6]:
# Install W&B 
# !pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: r1ck (use `wandb login --relogin` to force relogin)


True

In [7]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# 🦆 Hyperparameters

In [33]:
# TRAIN_PATH = 'input/siim-covid19-resized-to-256px-jpg/train/'
TRAIN_PATH = '../input/train/'

IMG_SIZE = 512 #256
BATCH_SIZE = 16
EPOCHS = 200

# 🔨 Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.

I am using [xhlulu's](https://www.kaggle.com/xhlulu) resized dataset. The uploaded 256x256 Kaggle dataset is [here](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg). Find other image resolutions [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918).

* Create train-validation split. <br>
* Create required `/dataset` folder structure and more the images to that folder. <br>
* Create `data.yaml` file needed to train the model. <br>
* Create bounding box coordinates in the required YOLO format. 

In [5]:
# !ls

In [34]:
# Everything is done from /kaggle directory.
# %cd ../

# Load image level csv file
df = pd.read_csv('../input/train_image_level.csv')

# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
# Add absolute path
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.png', axis=1)
# Get image level labels
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

df.head(5)

,id,boxes,label,StudyInstanceUID,path,image_level
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,../input/train/000a312787f2.png,opacity
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,../input/train/000c3a3f293f.png,none
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,../input/train/0012ff7358bc.png,opacity
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,../input/train/001398f4ff4f.png,opacity
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,../input/train/001bd15d1891.png,opacity


In [7]:
# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
# meta_df = pd.read_csv('input/siim-covid19-resized-to-256px-jpg/meta.csv')
# meta_df = pd.read_csv('../input/meta.csv')

# train_meta_df = meta_df.loc[meta_df.split == 'train']
# train_meta_df = train_meta_df.drop('split', axis=1)
# train_meta_df.columns = ['id', 'dim0', 'dim1']

# train_meta_df.head(2)

In [8]:
# # Merge both the dataframes
# df = df.merge(train_meta_df, on='id',how="left")
# df.head()

In [9]:
# df.columns

In [35]:
# df.to_csv('../input/train_v1.csv', index=False)

In [36]:
df = pd.read_csv('../input/train_v1.csv')

In [101]:
#Numeric Label and removed _study
train_study_level = pd.read_csv("../input/train_study_level.csv")
train_study_level

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1
...,...,...,...,...,...
6049,ffcb4630f46f_study,0,1,0,0
6050,ffe4d6e8fbb0_study,0,1,0,0
6051,ffe94fcb14fa_study,0,1,0,0
6052,ffebf1ef4a9c_study,0,1,0,0


In [102]:
# train_study_level.id = train_study_level.id.str[:-6]
train_study_level["class_id"] = train_study_level.apply(lambda l: np.argmax(l.values[1:]),axis=1)
train_study_level.rename(columns={'id':'study_id'}, inplace=True)
train_study_level

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id
0,00086460a852_study,0,1,0,0,1
1,000c9c05fd14_study,0,0,0,1,3
2,00292f8c37bd_study,1,0,0,0,0
3,005057b3f880_study,1,0,0,0,0
4,0051d9b12e72_study,0,0,0,1,3
...,...,...,...,...,...,...
6049,ffcb4630f46f_study,0,1,0,0,1
6050,ffe4d6e8fbb0_study,0,1,0,0,1
6051,ffe94fcb14fa_study,0,1,0,0,1
6052,ffebf1ef4a9c_study,0,1,0,0,1


In [103]:
train_study_level['Negative for Pneumonia'].value_counts()

0    4378
1    1676
Name: Negative for Pneumonia, dtype: int64

In [104]:
train_study_level['Typical Appearance'].value_counts()

0    3199
1    2855
Name: Typical Appearance, dtype: int64

In [105]:
train_study_level.class_id.value_counts()

1    2855
0    1676
2    1049
3     474
Name: class_id, dtype: int64

In [106]:
train_study_level

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id
0,00086460a852_study,0,1,0,0,1
1,000c9c05fd14_study,0,0,0,1,3
2,00292f8c37bd_study,1,0,0,0,0
3,005057b3f880_study,1,0,0,0,0
4,0051d9b12e72_study,0,0,0,1,3
...,...,...,...,...,...,...
6049,ffcb4630f46f_study,0,1,0,0,1
6050,ffe4d6e8fbb0_study,0,1,0,0,1
6051,ffe94fcb14fa_study,0,1,0,0,1
6052,ffebf1ef4a9c_study,0,1,0,0,1


In [107]:
# train_study_level.to_csv('../input/train_classification_v1.csv', index=False)

In [43]:
df.image_level.value_counts()

opacity    4294
none       2040
Name: image_level, dtype: int64

In [44]:
df

,id,boxes,label,StudyInstanceUID,path,image_level
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,../input/train/000a312787f2.png,opacity
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,../input/train/000c3a3f293f.png,none
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,../input/train/0012ff7358bc.png,opacity
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,../input/train/001398f4ff4f.png,opacity
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,../input/train/001bd15d1891.png,opacity
...,...,...,...,...,...,...
6329,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,../input/train/ffcc6edd9445.png,none
6330,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,../input/train/ffd91a2c4ca0.png,none
6331,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,../input/train/ffd9b6cf2961.png,opacity
6332,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,../input/train/ffdc682f7680.png,opacity


In [61]:
# df_2 = pd.merge(df, train_study_level, left_on='StudyInstanceUID', right_on='study_id', how='outer')
df_2 = pd.merge(train_study_level, df, left_on='study_id', right_on='StudyInstanceUID', how='inner')

In [62]:
df_2

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id,id,boxes,label,StudyInstanceUID,path,image_level
0,00086460a852,0,1,0,0,1,65761e66de9f,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852,../input/train/65761e66de9f.png,opacity
1,000c9c05fd14,0,0,0,1,3,51759b5579bc,NaN,none 1 0 0 1 1,000c9c05fd14,../input/train/51759b5579bc.png,none
2,00292f8c37bd,1,0,0,0,0,f6293b1c49e2,NaN,none 1 0 0 1 1,00292f8c37bd,../input/train/f6293b1c49e2.png,none
3,005057b3f880,1,0,0,0,0,3019399c31f4,NaN,none 1 0 0 1 1,005057b3f880,../input/train/3019399c31f4.png,none
4,0051d9b12e72,0,0,0,1,3,bb4b1da810f3,"[{'x': 812.54698, 'y': 1376.41291, 'width': 62...",opacity 1 812.54698 1376.41291 1435.14793 1806...,0051d9b12e72,../input/train/bb4b1da810f3.png,opacity
...,...,...,...,...,...,...,...,...,...,...,...,...
6329,ffcb4630f46f,0,1,0,0,1,84ed5f7f71bf,"[{'x': 1721.27651, 'y': 974.09667, 'width': 12...",opacity 1 1721.27651 974.09667 2999.21998 2681...,ffcb4630f46f,../input/train/84ed5f7f71bf.png,opacity
6330,ffe4d6e8fbb0,0,1,0,0,1,e6215d0188e5,"[{'x': 364.93056, 'y': 870.04017, 'width': 731...",opacity 1 364.93056 870.04017 1096.13908 2053....,ffe4d6e8fbb0,../input/train/e6215d0188e5.png,opacity
6331,ffe94fcb14fa,0,1,0,0,1,7d27b1bb3987,"[{'x': 28.48292, 'y': 828.48474, 'width': 1116...",opacity 1 28.48292 828.48474 1145.01081 2296.7...,ffe94fcb14fa,../input/train/7d27b1bb3987.png,opacity
6332,ffebf1ef4a9c,0,1,0,0,1,52478e480a15,"[{'x': 425.81211, 'y': 424.86147, 'width': 528...",opacity 1 425.81211 424.86147 953.95118 1579.3...,ffebf1ef4a9c,../input/train/52478e480a15.png,opacity


In [63]:
df_2.class_id.isna().sum()

0

In [67]:
df_2.id.nunique()

6334

In [69]:
df_2.study_id.nunique()

6054

In [86]:
df_2[df_2.id.duplicated()]

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id,id,boxes,label,StudyInstanceUID,path,image_level


In [92]:
df_2[df_2['study_id']=='00f9e183938e']

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id,id,boxes,label,StudyInstanceUID,path,image_level
16,00f9e183938e,0,0,0,1,3,6534a837497d,NaN,none 1 0 0 1 1,00f9e183938e,../input/train/6534a837497d.png,none
17,00f9e183938e,0,0,0,1,3,74077a8e3b7c,"[{'x': 2175.24285, 'y': 1123.72368, 'width': 4...",opacity 1 2175.24285 1123.72368 2607.50603 162...,00f9e183938e,../input/train/74077a8e3b7c.png,opacity


In [99]:
duplicate_study_id = df_2[df_2.duplicated(['study_id'])]

In [100]:
duplicate_study_id

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id,id,boxes,label,StudyInstanceUID,path,image_level
17,00f9e183938e,0,0,0,1,3,74077a8e3b7c,"[{'x': 2175.24285, 'y': 1123.72368, 'width': 4...",opacity 1 2175.24285 1123.72368 2607.50603 162...,00f9e183938e,../input/train/74077a8e3b7c.png,opacity
26,0142feaef82f,0,0,1,0,2,f5451a98d684,NaN,none 1 0 0 1 1,0142feaef82f,../input/train/f5451a98d684.png,none
79,0369e0385796,0,1,0,0,1,ea2117b53323,NaN,none 1 0 0 1 1,0369e0385796,../input/train/ea2117b53323.png,none
136,061d2ddaddd1,0,0,1,0,2,67ea53f37acf,"[{'x': 1996.08742, 'y': 981.46708, 'width': 61...",opacity 1 1996.08742 981.46708 2614.45583 1726...,061d2ddaddd1,../input/train/67ea53f37acf.png,opacity
139,066b12d875eb,0,0,1,0,2,eea3a910fa9e,NaN,none 1 0 0 1 1,066b12d875eb,../input/train/eea3a910fa9e.png,none
...,...,...,...,...,...,...,...,...,...,...,...,...
6204,fa9ea207e240,0,1,0,0,1,b7f123e4992f,"[{'x': 279.76393, 'y': 281.41593, 'width': 109...",opacity 1 279.76393 281.41593 1374.15922 2069....,fa9ea207e240,../input/train/b7f123e4992f.png,opacity
6212,fae06a05819a,0,1,0,0,1,ba375ad92c5f,NaN,none 1 0 0 1 1,fae06a05819a,../input/train/ba375ad92c5f.png,none
6242,fc45007f145a,0,1,0,0,1,8a38c00672d5,NaN,none 1 0 0 1 1,fc45007f145a,../input/train/8a38c00672d5.png,none
6275,fd92c6f2b2e6,1,0,0,0,0,e6cc65d9de1d,NaN,none 1 0 0 1 1,fd92c6f2b2e6,../input/train/e6cc65d9de1d.png,none


In [97]:
duplicate_study_id[duplicate_study_id.image_level=='none']

,study_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,class_id,id,boxes,label,StudyInstanceUID,path,image_level
26,0142feaef82f,0,0,1,0,2,f5451a98d684,NaN,none 1 0 0 1 1,0142feaef82f,../input/train/f5451a98d684.png,none
79,0369e0385796,0,1,0,0,1,ea2117b53323,NaN,none 1 0 0 1 1,0369e0385796,../input/train/ea2117b53323.png,none
139,066b12d875eb,0,0,1,0,2,eea3a910fa9e,NaN,none 1 0 0 1 1,066b12d875eb,../input/train/eea3a910fa9e.png,none
218,0949b847bda6,0,1,0,0,1,e60a47d36e62,NaN,none 1 0 0 1 1,0949b847bda6,../input/train/e60a47d36e62.png,none
309,0d33fb757a01,0,1,0,0,1,9b1de1c45491,NaN,none 1 0 0 1 1,0d33fb757a01,../input/train/9b1de1c45491.png,none
...,...,...,...,...,...,...,...,...,...,...,...,...
6119,f689f3fba17a,0,1,0,0,1,a323f1604ae2,NaN,none 1 0 0 1 1,f689f3fba17a,../input/train/a323f1604ae2.png,none
6126,f6ffe212deeb,1,0,0,0,0,21518ca15050,NaN,none 1 0 0 1 1,f6ffe212deeb,../input/train/21518ca15050.png,none
6212,fae06a05819a,0,1,0,0,1,ba375ad92c5f,NaN,none 1 0 0 1 1,fae06a05819a,../input/train/ba375ad92c5f.png,none
6242,fc45007f145a,0,1,0,0,1,8a38c00672d5,NaN,none 1 0 0 1 1,fc45007f145a,../input/train/8a38c00672d5.png,none


In [64]:
df_2.class_id.value_counts()

1    3007
0    1736
2    1108
3     483
Name: class_id, dtype: int64

In [78]:
# df_2.to_csv('../input/train_v1.csv', index=False)

## 🍘 Train-validation split

In [13]:
# Create train and validation split.
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df.image_level.values)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)

/home/rick/.pyenv/versions/3.8.2/envs/dl_pytorch/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/rick/.pyenv/versions/3.8.2/envs/dl_pytorch/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [14]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

Size of dataset: 6334, training images: 5067. validation images: 1267


## 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is: 

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `covid`.

In [15]:
os.makedirs('../tmp/covid/images/train', exist_ok=True)
os.makedirs('../tmp/covid/images/valid', exist_ok=True)

os.makedirs('../tmp/covid/labels/train', exist_ok=True)
os.makedirs('../tmp/covid/labels/valid', exist_ok=True)

! ls tmp/covid/images

ls: cannot access 'tmp/covid/images': No such file or directory


In [16]:
# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'../tmp/covid/images/train/{row.id}.jpg')
    else:
        copyfile(row.path, f'../tmp/covid/images/valid/{row.id}.jpg')

100%|██████████| 6334/6334 [00:07<00:00, 884.56it/s]


## 🍜 Create `.YAML` file

The `data.yaml`, is the dataset configuration file that defines 

1. an "optional" download command/URL for auto-downloading, 
2. a path to a directory of training images (or path to a *.txt file with a list of training images), 
3. a path to a directory of validation images (or path to a *.txt file with a list of validation images), 
4. the number of classes, 
5. a list of class names.

> 📍 Important: In this competition, each image can either belong to `opacity` or `none` image-level labels. That's why I have  used the number of classes, `nc` to be 2. YOLOv5 automatically handles the images without any bounding box coordinates. 

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required. 

In [17]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '../../tmp/covid/images/train',
    val = '../../tmp/covid/images/valid',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('../src/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat ../src/yolov5/data/data.yaml

{names: [none, opacity], nc: 2, train: ../../tmp/covid/images/train, val: ../../tmp/covid/images/valid}


## 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with **bounding box(es)** a `.txt` file with the same name as the image will be created in the format shown below:

* One row per object. <br>
* Each row is class `x_center y_center width height format`. <br>
* Box coordinates must be in normalized xywh format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by image width, and `y_center` and `height` by image height. <br>
* Class numbers are zero-indexed (start from 0). <br>

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets. 

In [18]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [19]:
# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.id
    # Get split
    split = row.split
    # Get image-level label
    label = row.image_level
    
    if row.split=='train':
        file_name = f'../tmp/covid/labels/train/{row.id}.txt'
    else:
        file_name = f'../tmp/covid/labels/valid/{row.id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|██████████| 6334/6334 [00:02<00:00, 2530.20it/s]


# 🚅 Train with W&B



In [20]:
%cd ../src/yolov5/

/home/rick/Dropbox/python_projects/data_science/Kaggle/siim_covid19/src/yolov5


```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5s.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [ ]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project kaggle-siim-covid

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-128-g4b52e19 torch 1.7.1+cu101 CUDA:0 (GeForce RTX 2080 Ti, 11019.0625MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='./data/data.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[512, 512], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='kaggle-siim-covid', quad=False, rect=False, resume=False, save_dir='kaggle-siim-covid/exp3', save_period=1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir kaggle-siim-covid', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_e

Saving model artifact on epoch  6

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     6/199     2.05G   0.05249   0.02022 0.0002141   0.07292        34       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.536       0.396       0.394       0.117
Saving model artifact on epoch  7

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     7/199     2.05G   0.05268    0.0207 0.0001859   0.07357        29       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.433       0.449       0.372       0.112
Saving model artifact on epoch  8

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
     8/199     2.05G   0.05212   0.02048 0.0001715   0.07278        21       512
               Class      Images      Labels           P     

    28/199     2.05G   0.04924   0.01964 9.083e-05   0.06896        22       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.505       0.497       0.439       0.129
Saving model artifact on epoch  29

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    29/199     2.05G   0.04885   0.01931 8.978e-05   0.06825        23       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.566       0.443       0.439        0.13
Saving model artifact on epoch  30

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    30/199     2.05G   0.04892    0.0197 8.379e-05    0.0687        26       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.537       0.461       0.445       0.131
S

    72/199     2.05G   0.04508   0.01907 5.777e-05   0.06422        25       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.506       0.467       0.432       0.131
Saving model artifact on epoch  73

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    73/199     2.05G   0.04458   0.01913 5.697e-05   0.06376        36       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.496       0.485       0.432       0.134
Saving model artifact on epoch  74

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
    74/199     2.05G   0.04446   0.01905 5.682e-05   0.06357        24       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.543       0.453       0.427       0.128
S

   116/199     2.05G   0.03932   0.01727 4.324e-05   0.05664        19       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.478       0.457       0.381       0.111
Saving model artifact on epoch  117

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   117/199     2.05G   0.03886   0.01719 4.434e-05    0.0561        32       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.469       0.473       0.377       0.111
Saving model artifact on epoch  118

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   118/199     2.05G   0.03868   0.01758 4.241e-05    0.0563        20       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.473       0.472       0.382       0.113

   138/199     2.05G   0.03611   0.01643 3.631e-05   0.05258        24       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.474       0.452       0.372       0.108
Saving model artifact on epoch  139

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   139/199     2.05G   0.03579    0.0164 3.528e-05   0.05223        21       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.451       0.469       0.366       0.108
Saving model artifact on epoch  140

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   140/199     2.05G   0.03593   0.01606 3.598e-05   0.05203        23       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.461        0.46       0.365       0.107

   160/199     2.05G   0.03306   0.01515 3.084e-05   0.04825        29       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.444       0.464       0.346      0.0993
Saving model artifact on epoch  161

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   161/199     2.05G   0.03284    0.0152 3.199e-05   0.04807        27       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.454       0.454       0.344      0.0995
Saving model artifact on epoch  162

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   162/199     2.05G   0.03276   0.01519 3.133e-05   0.04798        21       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.481       0.423        0.34      0.0982

   182/199     2.05G    0.0305   0.01389 2.774e-05   0.04442        33       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.486       0.429       0.337      0.0965
Saving model artifact on epoch  183

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   183/199     2.05G   0.03002    0.0141 2.649e-05   0.04414        22       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.481       0.436       0.335      0.0962
Saving model artifact on epoch  184

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
   184/199     2.05G   0.03031   0.01399 2.595e-05   0.04433        23       512
               Class      Images      Labels           P           R      mAP@.5
                 all        1267        1582       0.482       0.433       0.336      0.0959

## Model Saved Automatically as Artifact

Since it's a kernel based competition, you can easily download the best model from the W&B Artifacts UI and upload as a Kaggle dataset that you can load in your inference kernel (internel disabled).

### [Path to saved model $\rightarrow$](https://wandb.ai/ayush-thakur/kaggle-siim-covid/artifacts/model/run_jbt74n7q_model/4c3ca5752dba99bd227e)

![img](https://i.imgur.com/KhRLQvR.png)

> 📍 Download the model with the `best` alias tagged to it. 

# Inference

You will probably use a `Submission.ipynb` kernel to run all the predictions. After training a YOLOv5 based object detector -> head to the artifacts page and download the best model -> upload the model as a Kaggle dataset -> Use it with the submission folder. 

> 📍 Note that you might have to clone the YOLOv5 repository in a Kaggle dataset as well. 

In this section, I will show you how you can do the inference and modify the predicted bounding box coordinates.

In [ ]:
!ls

In [ ]:
# TEST_PATH = '/kaggle/input/siim-covid19-resized-to-256px-jpg/test/' # absolute path
TEST_PATH = '../input/test/' # absolute path

Since I am training the model in this kernel itself, I will not be using the method that I have described above. The best model is saved in the directory `project_name/exp*/weights/best.pt`. In `exp*`, * can be 1, 2, etc. 

In [ ]:
# MODEL_PATH = 'kaggle-siim-covid/exp/weights/best.pt'
MODEL_PATH = '../models/exp1/best.pt'

```
--weights {MODEL_PATH} \ # path to the best model.
--source {TEST_PATH} \ # absolute path to the test images.
--img {IMG_SIZE} \ # Size of image
--conf 0.281 \ # Confidence threshold (default is 0.25)
--iou-thres 0.5 \ # IOU threshold (default is 0.45)
--max-det 3 \ # Number of detections per image (default is 1000) 
--save-txt \ # Save predicted bounding box coordinates as txt files
--save-conf # Save the confidence of prediction for each bounding box
```

In [ ]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

### How to find the confidence score?

1. First first the [W&B run page](https://wandb.ai/ayush-thakur/kaggle-siim-covid/runs/jbt74n7q) generated by training the YOLOv5 model. 

2. Go to the media panel -> click on the F1_curve.png file to get a rough estimate of the threshold -> go to the Bounding Box Debugger panel and interactively adjust the confidence threshold. 

![img](https://i.imgur.com/cCUnTBw.gif)

> 📍 The bounding box coordinates are saved as text file per image name. It is saved in this directory `runs/detect/exp3/labels`. 

In [ ]:
PRED_PATH = 'runs/detect/exp3/labels'
!ls {PRED_PATH}

In [ ]:
# Visualize predicted coordinates.
%cat runs/detect/exp3/labels/ba91d37ee459.txt

> 📍 Note: 1 is class id (opacity), the first four float numbers are `x_center`, `y_center`, `width` and `height`. The final float value is `confidence`.

In [ ]:
prediction_files = os.listdir(PRED_PATH)
print('Number of test images predicted as opaque: ', len(prediction_files))

> 📍 Out of 1263 test images, 583 were predicted with `opacity` label and thus we have that many prediction txt files.

# Submission

In this section, I will show how you can use YOLOv5 as object detector and prepare `submission.csv` file.

In [ ]:
# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
def correct_bbox_format(bboxes):
    correct_bboxes = []
    for b in bboxes:
        xc, yc = int(np.round(b[0]*IMG_SIZE)), int(np.round(b[1]*IMG_SIZE))
        w, h = int(np.round(b[2]*IMG_SIZE)), int(np.round(b[3]*IMG_SIZE))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, xmax, ymin, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes

In [ ]:
# Read the submisison file
# sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
sub_df = pd.read_csv('../input/sample_submission.csv')
sub_df.tail()

In [ ]:
# Prediction loop for submission
predictions = []

for i in tqdm(range(len(sub_df))):
    row = sub_df.loc[i]
    id_name = row.id.split('_')[0]
    id_level = row.id.split('_')[-1]
    
    if id_level == 'study':
        # do study-level classification
        predictions.append("Negative 1 0 0 1 1") # dummy prediction
        
    elif id_level == 'image':
        # we can do image-level classification here.
        # also we can rely on the object detector's classification head.
        # for this example submisison we will use YOLO's classification head. 
        # since we already ran the inference we know which test images belong to opacity.
        if f'{id_name}.txt' in prediction_files:
            # opacity label
            confidence, bboxes = get_conf_bboxes(f'{PRED_PATH}/{id_name}.txt')
            bboxes = correct_bbox_format(bboxes)
            pred_string = ''
            for j, conf in enumerate(confidence):
                pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
            predictions.append(pred_string[:-1]) 
        else:
            predictions.append("None 1 0 0 1 1")

In [ ]:
sub_df['PredictionString'] = predictions
sub_df.to_csv('submission.csv', index=False)
sub_df.tail()

# WORK IN PROGRESS

Final component is model prediction visualization which is an optional debugging tool I would like to share. :)

Consider upvoting if you find the work useful. 